# Example of Using AnalyticFisher Library
### This notebook demonstrates calculating the Fisher Information Matrix for some simple PyTorch examples

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from fisher import *
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose

In [2]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

train_loader = DataLoader(training_data, batch_size=100)
test_loader = DataLoader(test_data, batch_size=100)

#### For a Simple 2 Layer Model

In [3]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define model
class SimpleModel(nn.Module):
    def __init__(self):
        super(SimpleModel, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    #Remember to include a function in the model that returns the last layer of weights named last_layer
    def last_layer(self):
        return self.Linear;

model = SimpleModel().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [4]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
            
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [5]:
epochs = 1
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_loader, model, loss_fn, optimizer)
    test(test_loader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.298291  [    0/60000]
loss: 2.280680  [10000/60000]
loss: 2.278570  [20000/60000]
loss: 2.259229  [30000/60000]
loss: 2.249781  [40000/60000]
loss: 2.229616  [50000/60000]
Test Error: 
 Accuracy: 38.1%, Avg loss: 2.220832 

Done!


In [14]:
#Define the Name of the Last Layer
weightname = "linear_relu_stack.2.weight"
for name, param in model.named_parameters():
    if name==weightname:
        print(name, param)

linear_relu_stack.2.weight Parameter containing:
tensor([[ 0.0043, -0.0232, -0.0043,  ...,  0.0420, -0.0249, -0.0289],
        [ 0.0206, -0.0330,  0.0273,  ...,  0.0318,  0.0319, -0.0156],
        [ 0.0037, -0.0018,  0.0085,  ..., -0.0203, -0.0349,  0.0095],
        ...,
        [-0.0015,  0.0404, -0.0321,  ..., -0.0417, -0.0271, -0.0117],
        [ 0.0397,  0.0355,  0.0049,  ...,  0.0354,  0.0410,  0.0269],
        [ 0.0141,  0.0071, -0.0111,  ..., -0.0283,  0.0018,  0.0224]],
       requires_grad=True)


In [15]:
CalcFIM(model, train_loader, 1, "linear_relu_stack.2.weight")

Calculating 1 of the Fisher...


  0%|          | 0/1 [00:00<?, ?it/s]

MemoryError: Unable to allocate 256. GiB for an array with shape (262144, 262144) and data type float32